In [807]:
from typing import Callable, List, Tuple, Union

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import copy

print("Finished importing packages")

Finished importing packages


## Systems of *nonlinear* equations in $\mathbb{R}^n$

Consider a $pT$-flash unit (compare Lecture 2). We are attempting to solve the following system of nonlinear equations:
\begin{equation}
\end{equation}

In [808]:
# define parameters:
T = 360.85 # 450 #K
p = 1 # 5 #bar

# Antoine parameters for component A, for p in bar and T in K
A_a = 5.15853 # 4.92531	
B_a = 1569.613 # 1432.526 #K
C_a = -34.846 # -61.819 #K

A_b = 3.55959	
B_b = 643.748
C_b = -198.043

# input variables
zA = 0.4 # 0.1
zB = 1-zA

In [809]:
def Antoine(T:float, A:float, B:float, C:float) -> float:
    """ 
    """
    p_sat = 10**((A-B/(T+C)))
    return p_sat

K_a = Antoine(T,A_a,B_a,C_a)/p
K_b = Antoine(T,A_b,B_b,C_b)/p

In [810]:
# Define the jacobian of our system

def jacobian(x:np.ndarray) -> np.ndarray:
    """
    """
    vf = x[0]
    lf = x[1]
    x_a =x[2]
    x_b = x[3] 

    jacobian_xk = [
            [K_a*x_a, x_a, vf*K_a+lf, 0],
            [K_b*x_b, x_b, vf*K_b+lf, 0],
            [0, 0, 1, 1],
            [1,1,0,0],
    ]
    jacobian_xk= np.array(jacobian_xk)
    return jacobian_xk

# Define the system of equations
def function_value(x:np.ndarray) -> np.ndarray:
    """
    """
    vf = x[0]
    lf = x[1]
    x_a =x[2]
    x_b = x[3]

    func_xk=[
        vf*K_a*x_a+lf*x_a-zA,
        vf*K_b*x_b+lf*x_b-zB,
        x_a+x_b-1,
        vf+lf-1
    ]
    func_xk = np.array(func_xk)
    return func_xk

In [811]:
def newtonRaphson(x_start:np.ndarray, nIter:int) -> np.ndarray:
    """ 
    """
    x_new = x_start
    for iter in range(nIter):
        x_prev = copy.deepcopy(x_new)
        x_new = x_new-np.linalg.inv(jacobian(x_prev))@function_value(x_prev)
        if np.allclose(x_new, x_prev, rtol = 1e-15, atol = 1e-15):
            print("Newton-Raphson converged after %i iterations."%iter)
            break
    return x_new


In [812]:
n = 4
x_start = np.array([0.5,0.5,0.3,0.7])
nIter = 200
# built-in scipy function: scipy.optimize.fsolve
x_root = scipy.optimize.root(function_value, x_start, jac = jacobian)

x_NetwonRaphson = newtonRaphson(x_start, nIter)
print(x_root.x)

print("Absolute error between scipy.optimixe.root and Newton-Raphson: {:.2e}\n\n".format(np.linalg.norm(x_NetwonRaphson -x_root.x)))


Newton-Raphson converged after 185 iterations.
[0.17320307 0.82679693 0.33083107 0.66916893]
Absolute error between scipy.optimixe.root and Newton-Raphson: 5.89e-12


